In [1]:
from PIL import Image
import os
import json
from tqdm import tqdm

In [2]:
train_pos_anno = "F:/Skripsi/deepfashion2_dataset_filter/train/pos/anno"
train_neg_anno = "F:/Skripsi/deepfashion2_dataset_filter/train/neg/anno"
train_pos_img = "F:/Skripsi/deepfashion2_dataset_filter/train/pos/img"
train_neg_img = "F:/Skripsi/deepfashion2_dataset_filter/train/neg/img"
train_pos_yolo_anno = "F:/Skripsi/deepfashion2_dataset_filter/train/pos/label"
train_neg_yolo_anno = "F:/Skripsi/deepfashion2_dataset_filter/train/neg/label"

In [3]:
def convert_to_yolo_format(annotation, image_width, image_height):
    bounding_box = annotation['bounding_box']
    category_id = annotation['category_id']
    
    x_min, y_min, x_max, y_max = bounding_box
    
    x_center = ((x_max + x_min) / 2) / image_width
    y_center = ((y_max + y_min) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    
    return f"{category_id} {x_center} {y_center} {width} {height}"

def process_directory(src_anno_dir, dest_dir, img_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for filename in tqdm(os.listdir(src_anno_dir), desc=f"Processing {src_anno_dir}"):
        if filename.endswith('.json'):
            with open(os.path.join(src_anno_dir, filename), 'r') as f:
                data = json.load(f)
            
            img_filename = os.path.splitext(filename)[0] + '.jpg'
            img_path = os.path.join(img_dir, img_filename)
        
            img = Image.open(img_path)
            image_width, image_height = img.size
            img.close()
        
            yolo_lines = []
            for key, value in data.items():
                if 'item' in key:
                    yolo_line = convert_to_yolo_format(value, image_width, image_height)
                    yolo_lines.append(yolo_line)
            
            txt_filename = os.path.splitext(filename)[0] + '.txt'
            with open(os.path.join(dest_dir, txt_filename), 'w') as f:
                f.write('\n'.join(yolo_lines))

#Execution
process_directory(train_pos_anno, train_pos_yolo_anno, train_pos_img)
process_directory(train_neg_anno, train_neg_yolo_anno, train_neg_img)

Processing F:/Skripsi/deepfashion2_dataset_filter/train/pos/anno: 100%|████████| 129560/129560 [40:25<00:00, 53.41it/s]
Processing F:/Skripsi/deepfashion2_dataset_filter/train/neg/anno: 100%|██████████| 71654/71654 [20:21<00:00, 58.67it/s]
